<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
from os import getenv
import psycopg2
from bs4 import BeautifulSoup as bs
import requests

In [2]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 
params = {
    "dbname": getenv('SF_DB_DBNAME'),
    "user": getenv('SF_DB_USER'),
    'host': getenv('SF_DB_HOST'),
    'password': getenv('SF_DB_PASSWORD'),
    'port': getenv('SF_DB_PORT'),
}

def df_from_query(query):
    with psycopg2.connect(**params) as connection:
        with connection.cursor() as cursor:
            cursor.execute(query)
            data = cursor.fetchall()
            columns = [x[0] for x in cursor.description]
            return pd.DataFrame(data, columns=columns)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [3]:
# текст запроса
query_3_1 = '''
    select count(*) from vacancies;
'''

In [4]:
result_3_1 = df_from_query(query_3_1)
print('Количество вакансий в базе:', result_3_1['count'][0])

Количество вакансий в базе: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [5]:
# текст запроса
query_3_2 = '''
    select count(*) from employers;
'''

In [6]:
# результат запроса
result_3_2 = df_from_query(query_3_2)
print('Количество работодателей в базе:', result_3_2['count'][0])

Количество работодателей в базе: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [7]:
# текст запроса
query_3_3 = '''
    select count(*) from areas;
'''

In [8]:
# результат запроса
result_3_3 = df_from_query(query_3_3)
print('Количество регионов в базе:', result_3_3['count'][0])

Количество регионов в базе: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [9]:
# текст запроса
query_3_4 = '''
    select count(*) from industries;
'''

In [10]:
# результат запроса
result_3_4 = df_from_query(query_3_4)
print('Количество отраслей в базе:', result_3_4['count'][0])

Количество отраслей в базе: 294


***

In [11]:
# выводы по предварительному анализу данных
print(
    f'''
        В базе данных содержится около {round(result_3_1['count'][0], -4):_} вакансий 
        от {result_3_2['count'][0]:_} работодателей 
        из {result_3_3['count'][0]:_} регионов, 
        покрывающих {result_3_4['count'][0]:_} отрасли.
        В среднем это чуть больше двух вакансий на работодателя.
    '''
)


        В базе данных содержится около 50_000 вакансий 
        от 23_501 работодателей 
        из 1_362 регионов, 
        покрывающих 294 отрасли.
        В среднем это чуть больше двух вакансий на работодателя.
    


В базе данных содержится около 50 000 вакансий  
от 23 501 работодателей  
из 1 362 регионов,  
покрывающих 294 отрасли.  
В среднем это чуть больше двух вакансий на работодателя.  

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = '''
    select areas.name as area, count(*) as vacancies_cnt
    from vacancies
    join areas on vacancies.area_id = areas.id
    group by area
    order by 2 desc;
'''

In [13]:
# результат запроса
result_4_1 = df_from_query(query_4_1)
print('Регионы по количеству вакансий:')
display(result_4_1)

Регионы по количеству вакансий:


,area,vacancies_cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса
query_4_2 = '''
    select count(*) 
    from vacancies 
    where salary_from is not null 
    or salary_to is not null;
'''

In [15]:
# результат запроса
result_4_2 = df_from_query(query_4_2)
print('Количество вакансий с указанием зарплаты:', result_4_2['count'][0])

Количество вакансий с указанием зарплаты: 24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
query_4_3 = '''
    select 'Нижняя граница:' as bound, round(avg(salary_from)) as avg_salary
    from vacancies
    where salary_from is not null
    union all
    select 'Верхняя граница:' as bound, round(avg(salary_to)) as avg_salary
    from vacancies
    where salary_to is not null;
'''

In [17]:
# результат запроса
result_4_3 = df_from_query(query_4_3)
result_4_3['avg_salary'] = result_4_3['avg_salary'].astype('int')
print('Средние значения для нижней и верхней границы зарплатной вилки:')
result_4_3

Средние значения для нижней и верхней границы зарплатной вилки:


,bound,avg_salary
0,Верхняя граница:,110537
1,Нижняя граница:,71065


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = '''
    select schedule, employment, count(*) as vacancies_cnt
    from vacancies
    group by schedule, employment
    order by 3 desc;
'''

In [19]:
# результат запроса
result_4_4 = df_from_query(query_4_4)
print('Количество вакансий по графику работы и типу занятости:')
result_4_4

Количество вакансий по графику работы и типу занятости:


,schedule,employment,vacancies_cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# текст запроса
query_4_5 = '''
    select experience, count(*) as vacancies_cnt
    from vacancies
    group by experience
    order by 2;
'''

In [21]:
# результат запроса
result_4_5 = df_from_query(query_4_5)
print('Количество вакансий по требуемому опыту работы:')
result_4_5

Количество вакансий по требуемому опыту работы:


,experience,vacancies_cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

In [22]:
# выводы по детальному анализу вакансий
query_4_6 = '''
    select experience,
        round(cast(sum(
                case
                    when salary_from is null and salary_to is null then 1
                    else 0
                    end
            ) as float) / count(*) * 100) as no_salary_percent
    from vacancies
    group by experience
    order by 2;
'''
result_4_6 = df_from_query(query_4_6)
print('Доля неуказания зарплаты в зависимости от опыта:')
result_4_6

Доля неуказания зарплаты в зависимости от опыта:


,experience,no_salary_percent
0,Нет опыта,30.0
1,От 1 года до 3 лет,51.0
2,Более 6 лет,59.0
3,От 3 до 6 лет,61.0


Самое большое количество вакансий, ожидаемо, в столицах и самых крупных городах.    
Средняя зарплатная вилка: от 70 000 до 110 000 рублей.  
Подавляющее большинство работодателй ищут сотрудников на полную занятость. А полный день в таких вакансиях — основной рабочий график.  
Больше половины вакансий предполагают опыт от года до трёх.  
Зарплата не указана в половине вакансий. Доля неуказаных зарплат растёт с требуемым опытом. Учитывая, что зарплаты растут с опытом, а вакансий с опытом больше 6 лет мало, то предсказать самые высокие зарплаты будет сложнее всего.  

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [23]:
# текст запроса
query_5_1 = '''
    with a as (select employers.name, count(*) as vacancies_cnt
        from vacancies
        join employers on vacancies.employer_id = employers.id
        group by employers.name
        order by 2 desc
        limit 5)
    select name, vacancies_cnt
    from (select * from a limit 1) as b
    union all
    select name, vacancies_cnt
    from (select * from a offset 4 limit 1) as c;
'''

In [24]:
# результат запроса
result_5_1 = df_from_query(query_5_1)
print('Первое и пятое место по количеству вакансий среди работодателей:')
result_5_1

Первое и пятое место по количеству вакансий среди работодателей:


,name,vacancies_cnt
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [25]:
# текст запроса
query_5_2 = '''
    with av as (select a.id,
                       a.name               as area_name,
                       count(distinct v.id) as vacancies_cnt
                from areas as a
                    left join vacancies as v on v.area_id = a.id
                group by a.id, a.name)
    select av.area_name,
        count(distinct e.id) as employers_cnt,
        av.vacancies_cnt
    from av
        left join employers e on av.id = e.area
    group by av.area_name, av.vacancies_cnt
    order by 2 desc;
'''

In [26]:
# результат запроса
result_5_2 = df_from_query(query_5_2)
print('Количество работодателей и вакансий по регионам:')
display(result_5_2)
print('Регион с наибольшим количеством работодателей, но без вакансий:')
display(result_5_2[(result_5_2['vacancies_cnt'] == 0)].sort_values(by='employers_cnt', ascending=False).head(1))
# no_vacancies = result_5_2[(result_5_2['vacancies_cnt'] == 0)]
# display(no_vacancies[no_vacancies['employers_cnt'] == no_vacancies['employers_cnt'].max()])

Количество работодателей и вакансий по регионам:


,area_name,employers_cnt,vacancies_cnt
0,Москва,5864,5333
1,Санкт-Петербург,2217,2851
2,Минск,1115,2112
3,Алматы,721,1892
4,Екатеринбург,609,1698
...,...,...,...
1357,Урень,0,1
1358,Уруссу,0,0
1359,Урюпинск,0,1
1360,Усмань,0,0


Регион с наибольшим количеством работодателей, но без вакансий:


,area_name,employers_cnt,vacancies_cnt
8,Россия,410,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [27]:
# текст запроса
query_5_3 = '''
    select
        e.name as employer_name,
        count(distinct a.name) as areas_cnt
    from employers as e
    join vacancies as v on e.id = v.employer_id
    join areas as a on v.area_id = a.id
    group by e.name
    order by 2 desc;
'''

In [28]:
# результат запроса
result_5_3 = df_from_query(query_5_3)
print('Количество регионов, в которых размещены вакансии работодателя:')
result_5_3

Количество регионов, в которых размещены вакансии работодателя:


,employer_name,areas_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [29]:
# текст запроса
query_5_4 = '''
    select count(*)
    from employers as e
    left join employers_industries ei on e.id = ei.employer_id
    where industry_id is null;
'''

In [30]:
# результат запроса
result_5_4 = df_from_query(query_5_4)
print('Количество работодателей, не указавших сферу деятельности:', result_5_4['count'][0])

Количество работодателей, не указавших сферу деятельности: 8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [31]:
# текст запроса
query_5_5 = '''
    select e.name
    from employers as e
    join employers_industries as ei on e.id = ei.employer_id
    group by e.name
    having count(distinct ei.industry_id) = 4
    order by e.name
    limit 1 offset 2;
'''

In [32]:
# результат запроса
result_5_5 = df_from_query(query_5_5)
print('Третий по алфавиту работодатель из указавших четыре сферы деятельности:', result_5_5['name'][0])

Третий по алфавиту работодатель из указавших четыре сферы деятельности: 2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [33]:
# текст запроса
query_5_6 = '''
    select count(*)
    from employers as e
    join employers_industries as ei on e.id = ei.employer_id
    join industries as i on ei.industry_id = i.id
    where i.name = 'Разработка программного обеспечения';
'''

In [34]:
# результат запроса
result_5_6 = df_from_query(query_5_6)
print('Количество работодателей, указавших сферу деятельности «Разработка программного обеспечения»:', result_5_6['count'][0])

Количество работодателей, указавших сферу деятельности «Разработка программного обеспечения»: 3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [35]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
responce = requests.get(url)

if responce.status_code == 200:
    soup = bs(responce.content)
    table = soup.find('table', {'class': 'standard sortable'})
    cities = [row.find_all('td')[1].text.replace('[a]', '') for row in table.find_all('tr')[1:]]

In [36]:
# текст запроса
query_5_7 = f'''
    select a.name               as area_name,
        count(distinct v.id) as vacancies_cnt
    from areas as a
            join vacancies as v on v.area_id = a.id
            join employers as e on v.employer_id = e.id
    where e.name = 'Яндекс'
    and a.name in {tuple(cities)}
    group by a.name
    union all
    select 'Total', count(distinct v.id)
    from areas as a
            join vacancies as v on v.area_id = a.id
            join employers as e on v.employer_id = e.id
    where e.name = 'Яндекс'
    and a.name in {tuple(cities)}
    order by 2;
'''

In [37]:
# результат запроса
result_5_7 = df_from_query(query_5_7)
print('Количество вакансий в городах-милионниках, в которых размещены вакансии компании «Яндекс»:')
result_5_7

Количество вакансий в городах-милионниках, в которых размещены вакансии компании «Яндекс»:


,area_name,vacancies_cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

In [38]:
# выводы по анализу работодателей

query_5_8 = f'''
    select e.name              as employer,
        count(distinct a.name) as areas_cnt,
        count(distinct v.id)   as vacancies_cnt
    from areas as a
            join vacancies as v on v.area_id = a.id
            join employers as e on v.employer_id = e.id
    where a.name in {tuple(cities)}
    group by e.name
    order by areas_cnt desc, vacancies_cnt desc;;
'''
result_5_8 = df_from_query(query_5_8)
print('Количество вакансий работодателей в городах-миллионниках:')
display(result_5_8.head())

query_5_9 = '''
    select eg.employer_group,
        count(*)                          as vacancies_cnt,
        sum(
                case
                    when salary_from is null and salary_to is null then 1
                    else 0
                    end
            )                             as vacancies_no_salary_cnt,
        round(cast(sum(
                case
                    when salary_from is null and salary_to is null then 1
                    else 0
                    end
            ) as float) / count(*) * 100) as no_salary_percent
    from vacancies as v
            join (select e.id,
                        case
                            when 100 <= count(*) then 'очень крупные'
                            when 30 <= count(*) and count(*) < 100 then 'крупные'
                            when 5 <= count(*) and count(*) < 30 then 'средние'
                            else 'мелкие'
                            end as employer_group
                from vacancies as v
                            join employers e on e.id = v.employer_id
                group by e.id) as eg on eg.id = v.employer_id
    group by eg.employer_group
    order by 4 desc
    ;
'''
result_5_9 = df_from_query(query_5_9)
print('Доля неуказания зарплаты в зависимости от размера работодателя:')
result_5_9

Количество вакансий работодателей в городах-миллионниках:


,employer,areas_cnt,vacancies_cnt
0,Яндекс,16,485
1,Ростелеком,16,214
2,Первый Бит,16,104
3,Тинькофф,15,285
4,Softline,15,93


Доля неуказания зарплаты в зависимости от размера работодателя:


,employer_group,vacancies_cnt,vacancies_no_salary_cnt,no_salary_percent
0,крупные,6625,4830,73.0
1,средние,15811,9389,59.0
2,очень крупные,6875,3991,58.0
3,мелкие,19886,6914,35.0


По количеству вакансий и регионов, в которых они размещены, лидирует Яндекс (1933 вакансий и 181 регион). Он же лидер в регионах-милионниках.  
Похоже, в вакансиях запрещено указывать регионы верхнего уровня, а в профиле работодателя это возможно.  
Сфера деятельности не указана почти у трети работодателей.  
Самая высокая доля неуказанных запрлат у работодателей с количеством вакансий от 30 до 100 (73%). Самая низкая — до 5 вакансий (35%).  

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [39]:
# текст запроса
query_6_1 = '''
    select count(*)
    from vacancies
    where name ilike '%data%'
    or name ilike '%данн%';
'''

In [40]:
# результат запроса
result_6_1 = df_from_query(query_6_1)
print('Вакансий, имеющих отношение к данным:', result_6_1['count'][0])

Вакансий, имеющих отношение к данным: 1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [41]:
# текст запроса
ds_patterns = ['%data scientist%', '%data science%','%исследователь данных%', "%machine learning%", '%машинн%обучен%']
ds_filter = f"(name ilike any (array{ds_patterns}) or name like '%ML%') and name not ilike '%HTML%'"
query_6_2 = f'''
    select count(*)
    from vacancies
    where {ds_filter}
    and (name ilike '%junior%'
        or experience = 'Нет опыта'
        or employment = 'Стажировка');
'''

In [42]:
# результат запроса
result_6_2 = df_from_query(query_6_2)
print('Вакансий для начинающего дата-саентиста:', result_6_2['count'][0])

Вакансий для начинающего дата-саентиста: 51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [43]:
# текст запроса
query_6_3 = f'''
    select count(id)
    from vacancies
    where {ds_filter}
    and key_skills ilike any (array['%SQL%', '%postgres%']);
'''

In [44]:
# результат запроса
result_6_3 = df_from_query(query_6_3)
print('Вакансий дата-саентистов с навыками SQL или postgres:', result_6_3['count'][0])

Вакансий дата-саентистов с навыками SQL или postgres: 201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [45]:
# текст запроса
query_6_4 = f'''
    select count(id)
    from vacancies
    where {ds_filter}
    and (key_skills ilike '%Python%');
'''

In [46]:
# результат запроса
result_6_4 = df_from_query(query_6_4)
print('Вакансий дата-саентистов с навыками Python:', result_6_4['count'][0])

Вакансий дата-саентистов с навыками Python: 351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [47]:
# текст запроса
query_6_5 = f'''
    select round(avg(array_length(string_to_array(key_skills, '	'), 1)), 2)
    from vacancies
    where {ds_filter};
'''

In [48]:
# результат запроса
result_6_5 = df_from_query(query_6_5)
result_6_5['round'] = result_6_5['round'].astype('float')
print('Среднее количество навыков в вакансиях дата-саентистов:', result_6_5['round'][0])

Среднее количество навыков в вакансиях дата-саентистов: 6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [49]:
# текст запроса
query_6_6 = f'''
    select experience, round(avg(coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from)) / 2, 0) as salary_avg
    from vacancies
    where {ds_filter}
    and (salary_from is not null or salary_to is not null)
    group by experience;
'''

In [50]:
# результат запроса
result_6_6 = df_from_query(query_6_6)
result_6_6['salary_avg'] = result_6_6['salary_avg'].astype(int)
print('Средняя зарплата дата-саентиста по опыту работы:')
display(result_6_6)
print('Средняя зарплата дата-саентиста c опытом работы от 3 до 6 лет:', result_6_6['salary_avg'][2])

Средняя зарплата дата-саентиста по опыту работы:


,experience,salary_avg
0,Нет опыта,74643
1,От 1 года до 3 лет,139675
2,От 3 до 6 лет,243115


Средняя зарплата дата-саентиста c опытом работы от 3 до 6 лет: 243115


***

In [51]:
# выводы по предметному анализу

query_6_7 = f'''
    select count(*)
    from vacancies
    where {ds_filter};
'''
result_6_7 = df_from_query(query_6_7)
print('Всего вакансий для дата-саентистов:', result_6_7['count'][0])

query_6_8 = f'''
    select experience,
        round(avg(coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from)) / 2, 0) as salary_avg,
        count(*)                          as vacancies_cnt,
        round(cast(sum(
                case
                    when salary_from is null and salary_to is null then 1
                    else 0
                    end
            ) as float) / count(*) * 100) as no_salary_percent
    from vacancies
    where {ds_filter}
    group by experience
    order by 4;
'''
result_6_8 = df_from_query(query_6_8)
print('Доля неуказания зарплаты для data science-вакансий в зависимости от опыта:')
display(result_6_8)

Всего вакансий для дата-саентистов: 480
Доля неуказания зарплаты для data science-вакансий в зависимости от опыта:


,experience,salary_avg,vacancies_cnt,no_salary_percent
0,Нет опыта,74643,28,75.0
1,От 3 до 6 лет,243115,195,86.0
2,От 1 года до 3 лет,139675,223,87.0
3,Более 6 лет,None,34,100.0


В базе данных содержится информация о 1771 вакансиях, имеющих отношение к данным. 480 вакансий подходят дата-саентистам, 51 из которых подходит начинающим, 201 требуют навыка SQL или postgres, а 351 — Python. От дата-саентистов в среднем требуется обладание более чем шестью навыками. Опытный (3-6 лет) дата-саентист может рассчитывать в среднем на зарплату в 240к рублей. Менее опытный (1-3 года) — 140к. Без опыта — 75к Доля неуказания зарплаты в data science-вакнсиях значительно превышает средние значения по базе и колеблется в зависимости от опыта от 75% (нет опыта) до 100% (более 6 лет опыта).

# Общий вывод по проекту

In [52]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования

# Посмотрим, как распределяются зарплаты в вакансиях, в которых присутствуют такие же скилы, как у дата-саентистов.
# Для этого отберём самые популярные скилы:
query_7_0 = f'''
    select lower(skill) as skill,
        count(*) as vacancies_cnt
    from vacancies,
        unnest(string_to_array(vacancies.key_skills, '	')) as skill
    where {ds_filter}
    group by lower(skill)
    having count(*) > 30
    order by 2 desc;
'''
result_7_0 = df_from_query(query_7_0)
print('Самые популярные скилы в data science-вакансиях:')
display(result_7_0)

# сделаем из них паттерны для запроса:
skills_list = result_7_0['skill'].tolist()
skill_pattern_list = [f'%{skill}%' for skill in skills_list]

# и вставим в наш запрос с распределением зарплат и доли их указания:
query_7_1 = f'''
    select experience,
        count(*)                                                                                      as vacancies_cnt,
        round(avg(coalesce(salary_from, salary_to) + coalesce(salary_to, salary_from)) / 2 * 1.27, 0) as salary_avg,
        round(cast(sum(
                case
                    when salary_from is null and salary_to is null then 1
                    else 0
                    end
            ) as float) / count(*) * 100)                                                             as no_salary_percent
    from vacancies
    where key_skills ilike any (array{skill_pattern_list})
    group by experience
    order by 4;
'''
result_7_1 = df_from_query(query_7_1)
print('Распределение зарплат по опыту и доля неуказанных зарплат в вакансиях, в которых указаны такие же ключевые навыки, как у специалистов в data science:')
display(result_7_1)

Самые популярные скилы в data science-вакансиях:


,skill,vacancies_cnt
0,python,348
1,sql,191
2,machine learning,140
3,git,66
4,data science,64
5,математическая статистика,62
6,pytorch,61
7,pandas,55
8,data analysis,54
9,ml,49


Распределение зарплат по опыту и доля неуказанных зарплат в вакансиях, в которых указаны такие же ключевые навыки, как у специалистов в data science:


,experience,vacancies_cnt,salary_avg,no_salary_percent
0,Нет опыта,1797,70154,51.0
1,Более 6 лет,630,251300,61.0
2,От 1 года до 3 лет,11233,120377,62.0
3,От 3 до 6 лет,7285,204574,67.0


Топ-5 городов покрывают 29% всех вакансий. А к полной занятости относятся 94% вакансий.

Вакансий с опытом больше 6 лет меньше всего — 2,7%. В этой же группе самая высокая доля неуказания зарплаты.

Для того, чтобы создать модель рекомендующую вакансии клиентам, претендующим на позицию Data Scientist, нужно будет заполнить неуказанные зарплаты. Сделать это достаточно точно довольно сложно, учитывая, что в этой области доля неуказания зарплат очень высокая, а для опыта больше 6 лет зарплата не указана ни для одной вакансии.

Для предсказания зарплаты на позиции Data Science можно попробовать использовать данные о других вакансиях, где доля указанных зарплат выше. Это могут быть как все вакансии, так и вакансии из смежных областей. Например, можно использовать вакансии имеющие такие же ключевые навыки (статистику для которых я привёл выше), подобрав коэффициенты для каждой группы. Естесственно, это нужно делать с учётом географии и, возможно, других параметров.